In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout


In [2]:
# 1. Generate synthetic dataset
np.random.seed(42)
n_samples = 5000
categories = ['groceries', 'rent', 'utilities', 'entertainment', 'salary', 'subscription']
merchants = ['Walmart', 'Netflix', 'Spotify', 'Amazon', 'Landlord', 'Employer']

In [3]:
data = pd.DataFrame({
    'amount': np.random.uniform(5, 5000, size=n_samples),
    'category': np.random.choice(categories, size=n_samples),
    'merchant': np.random.choice(merchants, size=n_samples),
    'day_of_month': np.random.randint(1, 31, size=n_samples),
    'day_of_week': np.random.randint(0, 7, size=n_samples),
    'hour': np.random.randint(0, 24, size=n_samples),
    'balance': np.random.uniform(0, 10000, size=n_samples)
})

In [4]:
# 2. Create synthetic labels
data['bill_due'] = (data['category'].isin(['rent', 'utilities', 'subscription']) & (data['day_of_month'] < 5)).astype(int)
data['expense_class'] = LabelEncoder().fit_transform(data['category'])
data['low_balance'] = (data['balance'] < 500).astype(int)


In [5]:
# 3. Preprocess features
features = data[['amount', 'day_of_month', 'day_of_week', 'hour', 'balance']]
features = pd.get_dummies(features.join(pd.get_dummies(data['merchant'], prefix='mch')), drop_first=True)

scaler = StandardScaler()
X = scaler.fit_transform(features)
# Labels
y_bill_due = data['bill_due']
y_expense_class = data['expense_class']
y_low_balance = data['low_balance']


In [6]:
# 4. Split data
X_train, X_test, y1_train, y1_test, y2_train, y2_test, y3_train, y3_test = train_test_split(
    X, y_bill_due, y_expense_class, y_low_balance, test_size=0.2, random_state=42)

In [7]:
# 5. Model 1: Bill Reminders (binary classification)
model_bill = Sequential([
    Dense(64, input_dim=X.shape[1], activation='relu'),
    Dropout(0.3),
    Dense(32, activation='relu'),
    Dense(1, activation='sigmoid')
])
model_bill.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model_bill.fit(X_train, y1_train, epochs=10, batch_size=32, verbose=1)

c:\Users\loret\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 4s 5ms/step - accuracy: 0.9102 - loss: 0.3622
Epoch 2/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.9401 - loss: 0.1684
Epoch 3/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.9420 - loss: 0.1426
Epoch 4/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.9334 - loss: 0.1443
Epoch 5/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.9331 - loss: 0.1413
Epoch 6/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.9342 - loss: 0.1250
Epoch 7/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.9374 - loss: 0.1269
Epoch 8/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.9363 - loss: 0.1208
Epoch 9/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.9374 - loss: 0.1137
Epoch 10/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.9425 - loss: 0.1135


In [15]:
# Save model
save_model(model_bill, "model_bill.h5")

# Save scaler
scaler = StandardScaler()
scaler.fit(X)
joblib.dump(scaler, "scaler.pkl")

['scaler.pkl']

In [8]:
# 6. Model 2: Expense Classification (multiclass)
model_expense = Sequential([
    Dense(64, input_dim=X.shape[1], activation='relu'),
    Dropout(0.3),
    Dense(32, activation='relu'),
    Dense(len(np.unique(y_expense_class)), activation='softmax')
])
model_expense.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model_expense.fit(X_train, y2_train, epochs=10, batch_size=32, verbose=1)


Epoch 1/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - accuracy: 0.1725 - loss: 1.8445
Epoch 2/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.1693 - loss: 1.7956
Epoch 3/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.1945 - loss: 1.7956
Epoch 4/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.1895 - loss: 1.7870
Epoch 5/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.2080 - loss: 1.7862
Epoch 6/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.1960 - loss: 1.7820
Epoch 7/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.2051 - loss: 1.7782
Epoch 8/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.1985 - loss: 1.7813
Epoch 9/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.2224 - loss: 1.7707
Epoch 10/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.2039 - loss: 1.7779


In [14]:
# Save model
save_model(model_expense, "model_bill.h5")

# Save scaler
scaler = StandardScaler()
scaler.fit(X)
joblib.dump(scaler, "scaler.pkl")

['scaler.pkl']

In [9]:
# 7. Model 3: Low Balance Prediction (binary classification)
model_balance = Sequential([
    Dense(64, input_dim=X.shape[1], activation='relu'),
    Dropout(0.3),
    Dense(32, activation='relu'),
    Dense(1, activation='sigmoid')
])
model_balance.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model_balance.fit(X_train, y3_train, epochs=10, batch_size=32, verbose=1)

Epoch 1/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - accuracy: 0.7315 - loss: 0.4890
Epoch 2/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.9488 - loss: 0.1611
Epoch 3/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.9550 - loss: 0.1226
Epoch 4/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.9512 - loss: 0.1099
Epoch 5/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.9568 - loss: 0.0969
Epoch 6/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.9634 - loss: 0.0842
Epoch 7/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.9652 - loss: 0.0778
Epoch 8/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.9668 - loss: 0.0769
Epoch 9/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.9711 - loss: 0.0690
Epoch 10/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.9660 - loss: 0.0769


In [13]:
# Save model
save_model(model_balance, "model_bill.h5")

# Save scaler
scaler = StandardScaler()
scaler.fit(X)
joblib.dump(scaler, "scaler.pkl")

['scaler.pkl']

In [10]:
# 8. Evaluate each model
print("\nEvaluation - Bill Reminder:")
model_bill.evaluate(X_test, y1_test, verbose=1)

print("\nEvaluation - Expense Classification:")
model_expense.evaluate(X_test, y2_test, verbose=1)

print("\nEvaluation - Low Balance Alert:")
model_balance.evaluate(X_test, y3_test, verbose=1)



Evaluation - Bill Reminder:
32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.9259 - loss: 0.1145

Evaluation - Expense Classification:
32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.1414 - loss: 1.8182

Evaluation - Low Balance Alert:
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.9797 - loss: 0.0581


[0.0571393258869648, 0.9810000061988831]

In [ ]:
# finance_system.py

import datetime

# Sample data
bills = [
    {"name": "Electricity", "due_date": "2025-06-15", "amount": 100},
    {"name": "Internet", "due_date": "2025-06-20", "amount": 50},
]

expenses = [
    {"description": "Grocery shopping", "amount": 80},
    {"description": "Movie tickets", "amount": 30},
    {"description": "Bus fare", "amount": 10},
    {"description": "Restaurant", "amount": 60},
]

expense_categories = {
    "grocery": "Food",
    "movie": "Entertainment",
    "bus": "Transport",
    "restaurant": "Food",
}

balance = 200  # Synthetic balance

# 1. Bill Reminders
def bill_reminders(bills):
    today = datetime.date.today()
    print("Upcoming Bill Reminders:")
    for bill in bills:
        due = datetime.datetime.strptime(bill["due_date"], "%Y-%m-%d").date()
        days_left = (due - today).days
        if days_left <= 7:
            print(f"  - {bill['name']} bill of ${bill['amount']} is due in {days_left} days ({bill['due_date']})")

# 2. Expense Classification
def classify_expenses(expenses):
    print("\nExpense Classification:")
    for exp in expenses:
        desc = exp["description"].lower()
        category = "Other"
        for key in expense_categories:
            if key in desc:
                category = expense_categories[key]
                break
        print(f"  - {exp['description']}: {category}")

# 3. Low Balance Simulation
def simulate_balance(balance, expenses, bills):
    total_expenses = sum(e["amount"] for e in expenses)
    total_bills = sum(b["amount"] for b in bills)
    projected_balance = balance - total_expenses - total_bills
    print(f"\nProjected balance after expenses and bills: ${projected_balance}")
    if projected_balance < 50:
        print("Warning: Low balance! Consider reducing expenses.")

if __name__ == "__main__":
    bill_reminders(bills)
    classify_expenses(expenses)
    simulate_balance(balance, expenses, bills)

Upcoming Bill Reminders:
  - Electricity bill of $100 is due in -1 days (2025-06-15)
  - Internet bill of $50 is due in 4 days (2025-06-20)

Expense Classification:
  - Grocery shopping: Food
  - Movie tickets: Entertainment
  - Bus fare: Transport
  - Restaurant: Food

Projected balance after expenses and bills: $-130


In [12]:
from tensorflow.keras.models import save_model
import joblib

# Save models
save_model(model_bill, "model_bill.h5")
save_model(model_expense, "model_expense.h5")
save_model(model_balance, "model_balance.h5")

# Save scaler
joblib.dump(scaler, "scaler.pkl")


['scaler.pkl']

In [16]:
import tkinter as tk
from tkinter import messagebox
import datetime

In [17]:
# Sample data
bills = [
    {"name": "Electricity", "due_date": "2025-06-15", "amount": 100},
    {"name": "Internet", "due_date": "2025-06-20", "amount": 50},
    {"name": "Water", "due_date": "2025-06-18", "amount": 30},
]

expenses = [
    {"description": "Grocery shopping", "amount": 80},
    {"description": "Movie tickets", "amount": 30},
    {"description": "Bus fare", "amount": 10},
    {"description": "Restaurant", "amount": 60},
]

expense_categories = {
    "grocery": "Food",
    "movie": "Entertainment",
    "bus": "Transport",
    "restaurant": "Food",
}

balance = 200  # Synthetic balance

def get_upcoming_bills(bills):
    today = datetime.date.today()
    reminders = []
    for bill in bills:
        due = datetime.datetime.strptime(bill["due_date"], "%Y-%m-%d").date()
        days_left = (due - today).days
        if 0 <= days_left <= 7:
            reminders.append(f"{bill['name']} (${bill['amount']}) due in {days_left} days ({bill['due_date']})")
    return reminders

def show_bill_reminders():
    reminders = get_upcoming_bills(bills)
    if reminders:
        reminder_text = "\n".join(reminders)
    else:
        reminder_text = "No bills due in the next 7 days."
    messagebox.showinfo("Bill Reminders", reminder_text)

def classify_expenses(expenses):
    results = []
    for exp in expenses:
        desc = exp["description"].lower()
        category = "Other"
        for key in expense_categories:
            if key in desc:
                category = expense_categories[key]
                break
        results.append(f"{exp['description']}: {category} (${exp['amount']})")
    return results

def show_expense_classification():
    results = classify_expenses(expenses)
    if results:
        text = "\n".join(results)
    else:
        text = "No expenses to classify."
    messagebox.showinfo("Expense Classification", text)

def simulate_balance(balance, expenses, bills):
    total_expenses = sum(e["amount"] for e in expenses)
    total_bills = sum(b["amount"] for b in bills)
    projected_balance = balance - total_expenses - total_bills
    return projected_balance

def show_balance_warning():
    projected = simulate_balance(balance, expenses, bills)
    msg = f"Projected balance after expenses and bills: ${projected}"
    if projected < 50:
        msg += "\nWarning: Low balance! Consider reducing expenses."
    messagebox.showinfo("Balance Simulation", msg)

In [ ]:
# Tkinter GUI
import tkinter as tk
from tkinter import messagebox

root = tk.Tk()
root.title("Personal Finance Assistant")

frame = tk.Frame(root, padx=20, pady=20)
frame.pack()

label = tk.Label(frame, text="Personal Finance Assistant", font=("Arial", 14, "bold"))
label.pack(pady=10)

remind_button = tk.Button(frame, text="Show Bill Reminders", command=show_bill_reminders, font=("Arial", 12))
remind_button.pack(pady=5)

classify_button = tk.Button(frame, text="Classify Expenses", command=show_expense_classification, font=("Arial", 12))
classify_button.pack(pady=5)

balance_button = tk.Button(frame, text="Simulate Balance", command=show_balance_warning, font=("Arial", 12))
balance_button.pack(pady=5)

root.mainloop()

In [18]:
# --- Main App Interface ---
def finance_ui():
    login_window.destroy()
    root = tk.Tk()
    root.title("Personal Finance Assistant")

    frame = tk.Frame(root, padx=20, pady=20)
    frame.pack()

    label = tk.Label(frame, text="Personal Finance Assistant", font=("Arial", 14, "bold"))
    label.pack(pady=10)

    tk.Button(frame, text="Show Bill Reminders", command=show_bill_reminders, font=("Arial", 12)).pack(pady=5)
    tk.Button(frame, text="Classify Expenses", command=show_expense_classification, font=("Arial", 12)).pack(pady=5)
    tk.Button(frame, text="Simulate Balance", command=show_balance_warning, font=("Arial", 12)).pack(pady=5)

    root.mainloop()


In [ ]:
# --- Login Screen ---
def attempt_login():
    username = entry_user.get()
    password = entry_pass.get()
    if username == "admin" and password == "1234":
        finance_ui()
    else:
        messagebox.showerror("Login Failed", "Invalid username or password")

login_window = tk.Tk()
login_window.title("Login")

tk.Label(login_window, text="Username:").grid(row=0, column=0, padx=10, pady=10)
entry_user = tk.Entry(login_window)
entry_user.grid(row=0, column=1)

tk.Label(login_window, text="Password:").grid(row=1, column=0, padx=10, pady=10)
entry_pass = tk.Entry(login_window, show="*")
entry_pass.grid(row=1, column=1)

tk.Button(login_window, text="Login", command=attempt_login).grid(row=2, columnspan=2, pady=10)

login_window.mainloop()